In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from torch.autograd import Variable

In [2]:
model = models.resnet18(pretrained=True)

In [3]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [4]:
from PIL import Image
import numpy as np

In [5]:
img = Image.open('../cat.jpeg')

In [6]:
def preprocess(img):
    scale = transforms.Resize((224, 224))
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    tensor = transforms.ToTensor()
    return normalize(tensor(scale(img)))

In [7]:
var = Variable(preprocess(img).unsqueeze(0))

In [8]:
outputs = []

In [9]:
def copy(model, inp, out):
    outputs.append(out)

In [10]:
model.layer2[0].conv1.register_forward_hook(copy)

In [11]:
g = model(var)

In [15]:
outputs[0].size()

torch.Size([1, 128, 28, 28])

In [ ]:
result = g.data.numpy().reshape((64, 64))

In [ ]:
im = Image.fromarray(result, 'P')

In [ ]:
im.show()

In [ ]:
result

In [8]:
import numpy as np

class Kalman:
    def __init__(self, state_dim: int = 6, obs_dim: int = 2):
        self.state_dim      = state_dim
        self.obs_dim        = obs_dim
        
        self.noise          = np.matrix(np.eye(state_dim)*1e-4)
        self.observation    = np.matrix(np.eye(obs_dim)*0.01)
        self.transition     = np.matrix(np.eye(state_dim))
        self.measurement    = np.matrix(np.zeros((obs_dim, state_dim)))
        self.gain           = np.matrix(np.zeros_like(self.measurement.T))
        self.covariance     = np.matrix(np.zeros_like(self.transition))
        self.state          = np.matrix(np.zeros((state_dim, 1)))
    
        if obs_dim == int(state_dim/3):
            # x( t + 1 ) = x( t ) + v( t ) + a( t ) / 2
            idx = np.r_[0:obs_dim]
            self.measurement[np.ix_(idx,idx)]                   = np.eye(obs_dim)
            self.transition                                     = np.eye(state_dim)
            self.transition[np.ix_(idx,idx+obs_dim)]            += np.eye(obs_dim)
            self.transition[np.ix_(idx,idx+obs_dim*2)]          += 0.5 * np.eye(obs_dim)
            self.transition[np.ix_(idx+obs_dim,idx+obs_dim*2)]  += np.eye(obs_dim)
            
    def update(self, pos: np.ndarray):
        if pos.ndim == 1: pos = np.matrix(pos).T
        
        # make prediction
        self.state      = self.transition * self.state
        self.covariance = self.transition * self.covariance * self.transition.T + self.noise
        
        # compute optimal kalman gain factor
        self.gain       = self.covariance * self.measurement.T * np.linalg.inv(self.measurement * self.covariance * self.measurement.T + self.observation)
        
        # correction based on observation
        self.state      = self.state + self.gain * (pos - self.measurement * self.state)
        self.covariance = self.covariance - self.gain * self.measurement * self.covariance

    def predict(self) -> np.ndarray:
        return np.asarray(self.measurement * self.state)


In [9]:
k = Kalman()

In [10]:
k.update(np.r_[2, 3])

In [11]:
k.predict().reshape((2))

array([0.01980198, 0.02970297])